In [10]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: dataset/
   creating: dataset/fighting/
  inflating: dataset/fighting/10-miles-to-safety.png  
  inflating: dataset/fighting/10-second-ninja-x.png  
  inflating: dataset/fighting/100ft-robot-golf.png  
  inflating: dataset/fighting/3-count-bout.png  
  inflating: dataset/fighting/5-force-fighters.png  
  inflating: dataset/fighting/99vidas.png  
  inflating: dataset/fighting/a-dot-i-m-dot-dollars-all-you-need-is-money.png  
  inflating: dataset/fighting/a-gummys-life.png  
  inflating: dataset/fighting/abobos-big-adventure.png  
  inflating: dataset/fighting/abraca-imagic-games.png  
  inflating: dataset/fighting/absolver.png  
  inflating: dataset/fighting/ace-of-seafood.png  
  inflating: dataset/fighting/aces-of-the-luftwaffe-squadron.png  
  inflating: dataset/fighting/actraiser.png  
  inflating: dataset/fighting/adventure-time-pirates-of-the-enchiridion.png  
  inflating: dataset/fighting/age-of-gladiators.png  
  inflating: dataset/fighting/age

In [2]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [3]:
import torch
from PIL import Image,ImageFile
from transformers import AutoProcessor, LlavaForConditionalGeneration
import gc
import csv
import os
from tqdm import tqdm

In [4]:
gc.collect()
torch.cuda.empty_cache()

In [5]:

PROMPT = "Describe briefly this video game cover. Mention characters ,visible text, environment, objects, colors and style"
MODEL_NAME = "fancyfeast/llama-joycaption-beta-one-hf-llava"


In [34]:
# Load JoyCaption
# bfloat16 is the native dtype of the LLM used in JoyCaption (Llama 3.1)
# device_map=0 loads the model into the first GPU
processor = AutoProcessor.from_pretrained(MODEL_NAME)
llava_model = LlavaForConditionalGeneration.from_pretrained(MODEL_NAME, torch_dtype="float16", device_map="auto")
llava_model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

preprocessor_config.json:   0%|          | 0.00/393 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.16G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/219 [00:00<?, ?B/s]

LlavaForConditionalGeneration(
  (model): LlavaModel(
    (vision_tower): SiglipVisionModel(
      (vision_model): SiglipVisionTransformer(
        (embeddings): SiglipVisionEmbeddings(
          (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
          (position_embedding): Embedding(729, 1152)
        )
        (encoder): SiglipEncoder(
          (layers): ModuleList(
            (0-26): 27 x SiglipEncoderLayer(
              (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
              (self_attn): SiglipAttention(
                (k_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (v_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (q_proj): Linear(in_features=1152, out_features=1152, bias=True)
                (out_proj): Linear(in_features=1152, out_features=1152, bias=True)
              )
              (layer_norm2): LayerNorm((1152,), eps=1e-06, elementwise_

In [6]:
def open_image_safe(path):
    ImageFile.LOAD_TRUNCATED_IMAGES = True
    try:
        img = Image.open(path).convert("RGB")
        return img
    except Exception as e:
        print(f"Failed to open {path}: {e}")
        return None

In [52]:
DATASET_DIR = "dataset"

all_files = []
for root, dirs, files in os.walk(DATASET_DIR):
    for file in files:
        if file.lower().endswith(('.png', '.jpg', '.jpeg')):  # optional filter
            all_files.append(os.path.join(root, file))


In [57]:
OUTPUT_CSV = "captions.csv"
row_count = 0
if  os.path.isfile(OUTPUT_CSV):
  with open(OUTPUT_CSV,) as f:
    row_count = sum(1 for row in f)
    # remove header
    row_count -= 1
if row_count > len(all_files):
  row_count = 0
print(row_count)

3


In [56]:
results = []
checkpoint = 0
header = row_count == 0
# Build the conversation
convo = [
            {
                "role": "system",
                "content": "You are a helpful image captioner.",
            },
            {
                "role": "user",
                "content": PROMPT,
            },
        ]
# Format the conversation
        # WARNING: HF's handling of chat's on Llava models is very fragile.  This specific combination of processor.apply_chat_template(), and processor() works
        # but if using other combinations always inspect the final input_ids to ensure they are correct.  Often times you will end up with multiple <bos> tokens
        # if not careful, which can make the model perform poorly.
convo_string = processor.apply_chat_template(convo, tokenize = False, add_generation_prompt = True)
assert isinstance(convo_string, str)

with torch.no_grad():
  for image_path in tqdm(all_files[row_count:], desc="Processing images"):
        if not image_path.lower().endswith((".png", ".jpg", ".jpeg")):
            continue  # skip non-image files
        # Load image
        image = open_image_safe(image_path)
        if image is None:
          continue


        # Process the inputs
        inputs = processor(text=[convo_string], images=[image], return_tensors="pt").to('cuda')
        inputs['pixel_values'] = inputs['pixel_values'].to(torch.float16)

        # Generate the captions
        generate_ids = llava_model.generate(
            **inputs,
            max_new_tokens=77,
            do_sample=True,
            suppress_tokens=None,
            use_cache=True,
            temperature=0.6,
            top_k=None,
            top_p=0.9,
        )[0]

        # Trim off the prompt
        generate_ids = generate_ids[inputs['input_ids'].shape[1]:]

        # Decode the caption
        caption = processor.tokenizer.decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
        caption = caption.strip()

        print(f"{image_path}: {caption}")
        results.append([image_path, caption])
        checkpoint+=1
        if checkpoint == 1:
          checkpoint = 0
          with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
            writer = csv.writer(f)
            if header:
              writer.writerow(["image_path","caption"])  # header
              header = False
            writer.writerows(results)
          results = []
  # if loop doesn't finish on a 10
  if checkpoint != 10:
    with open(OUTPUT_CSV, "a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerows(results)



Processing images:   0%|          | 1/1998 [01:26<47:51:36, 86.28s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


dataset/platform/metal-slug.png: Digital illustration of a muscular, blonde man with a headband, grinning, holding a rifle. He's surrounded by various firearms, ammo, and grenades. He wears a red vest over a white shirt and tan pants. The background is black. Bold, orange text reads "METAL SLUG" at the bottom. The style is vibrant and comic-like, with strong colors and


Processing images:   0%|          | 1/1998 [01:33<51:44:32, 93.28s/it]


KeyboardInterrupt: 